In [1]:
%%init_spark
launcher.jars = ["/app/setup/commons.jar"]
launcher.conf.spark.app.name = "tej_scratch_3"
launcher.conf.spark.local.dir = "/data/tmp/spark" 
launcher.conf.spark.sql.shuffle.partitions = 200  
launcher.conf.spark.sql.shuffle.minPartitions = 20 
launcher.conf.spark.driver.memory = "8g" 
launcher.conf.spark.executor.memory = "3g" 
launcher.conf.spark.ui.showConsoleProgress = "true"
launcher.master = "local[5]"

In [2]:
var df_tq = spark.read.parquet("/data1/searchengine/EmbeddingTrainingDataCuration/ajio/09122024/V2/TemplatedQueries")

Intitializing Scala interpreter ...

Spark Web UI available at http://10.166.181.219:4050
SparkContext available as 'sc' (version = 3.3.0, master = local[5], app id = local-1739186214684)
SparkSession available as 'spark'


df_tq: org.apache.spark.sql.DataFrame = [query: array<string>, query_template: array<string> ... 6 more fields]


In [3]:
df_tq.show()

+--------------------+--------------------+------------------+--------------------+-------------------------+-------------------------+-------------+----------+
|               query|      query_template|template_frequency|        product_code|l1l2category_en_string_mv|l1l3category_en_string_mv|product_count|is_synonym|
+--------------------+--------------------+------------------+--------------------+-------------------------+-------------------------+-------------+----------+
|[, embellished, s...|[brickprimarycolo...|              1221|[q61d, q61q, q61g...|     [{9, Women - Ethn...|     [{9, Women - Sare...|            9|        no|
|[, satchels, hand...|[brickprimarycolo...|              1269|        [1a19, 1a18]|     [{2, Women - Bags...|     [{2, Women - Hand...|            2|        no|
|[, scarves, for, ...|[brickprimarycolo...|              4135|[18a8, 18a4, 18a9...|     [{8, Men - Clothi...|     [{8, Men - Scarves}]|            8|        no|
|[ american diamon...|      [name_

In [4]:
df_tq = df_tq.select("query")

df_tq: org.apache.spark.sql.DataFrame = [query: array<string>]


In [5]:
df_tq.show()

+--------------------+
|               query|
+--------------------+
|[, embellished, s...|
|[, satchels, hand...|
|[, scarves, for, ...|
|[ american diamon...|
|[ chunky heeled s...|
|[ double bed fold...|
|[ floral printed ...|
|[ henley t-shirt ...|
|[ set of 3 abstra...|
|[ women printed 2...|
|   [&circus, briefs]|
|['malie, western ...|
|[02 pink, lips, f...|
|[021-black, short...|
|[10 piece glass c...|
|[130 gsm geometri...|
|[144 tc floral pr...|
|[144 tc striped k...|
|[15.6" laptop mes...|
|[150 gsm king bed...|
+--------------------+
only showing top 20 rows



In [6]:
import scala.util.Random

var rand = new Random()

def add_space(query_list:List[String])={
    if(query_list.length>0){
    var index = rand.nextInt(query_list.length)
        if(query_list(index).length>0){
            var index_of_string = rand.nextInt(query_list(index).length)
            var new_string = query_list(index).substring(0,index_of_string)+" "+query_list(index).substring(index_of_string)
            var new_query_list = query_list.updated(index,new_string)
        new_query_list
        }else{
            query_list
        }
    }else{
        query_list
    }
}

var add_space_udf = udf(add_space _)

import scala.util.Random
rand: scala.util.Random = scala.util.Random@2bbcd2ec
add_space: (query_list: List[String])List[String]
add_space_udf: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$3931/437562393@1f836084,ArrayType(StringType,true),List(Some(class[value[0]: array<string>])),Some(class[value[0]: array<string>]),None,true,true)


In [7]:
df_tq.printSchema()

root
 |-- query: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [8]:
df_tq = df_tq.withColumn("bad_queries",add_space_udf(col("query")))

df_tq: org.apache.spark.sql.DataFrame = [query: array<string>, bad_queries: array<string>]


In [9]:
df_tq.printSchema()

root
 |-- query: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- bad_queries: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [10]:
df_tq = df_tq.withColumn("query",concat_ws(" ",col("query"))).withColumn("bad_queries",concat_ws(" ",col("bad_queries")))

df_tq: org.apache.spark.sql.DataFrame = [query: string, bad_queries: string]


In [11]:
df_tq = df_tq.filter(length(col("query")) <=25 )
df_tq =df_tq.sample(fraction=0.5).limit(2000)

df_tq: org.apache.spark.sql.DataFrame = [query: string, bad_queries: string]
df_tq: org.apache.spark.sql.DataFrame = [query: string, bad_queries: string]


In [12]:
df_tq.count()

res4: Long = 2000


In [14]:
df_tq.show(false)

+-------------------------+--------------------------+
|query                    |bad_queries               |
+-------------------------+--------------------------+
| embellished sarees      | e mbellished sarees      |
|&circus briefs           |&circus  briefs           |
|1g yellow gold om coin   |1g yellow  gold om coin   |
|574 lace-up casual shoes |574 lace-up casual sh oes |
|612 league casual tops   |612 league casual t ops   |
|a-line dresses in komarri|a-line dresses  in komarri|
|abdesigns                | abdesigns                |
|abstract shirts in kotty |abstract shir ts in kotty |
|abstract ties in cazzano |abstract ties in cazza no |
|acai occasion flat shoes |acai occasi on flat shoes |
|adamo timewear & eyewear |adamo timewear & eyew ear |
|adidas footwear for women|adidas footwear  for women|
|adobe tshirts for men    |adobe t shirts for men    |
|air garb tshirts for men |air garb tshir ts for men |
|airforce blue cotton sets|airforce blue cotton se ts|
|aj dezine

In [15]:
df_tq.write.option("header",true).parquet("file:////app/notebooks/avinash/SpellCheck-test-data/word_break_test_data.csv")

In [16]:
// saving to hdfs
df_tq.write.option("header",true).csv("/data1/archive/avinash/SearchTests/TestData/SpellCheck/word_break_test_data.csv")